In [1]:
import numpy as np
import time
from scipy.linalg import sqrtm

np.random.seed(13)

eps = 1e-31
d = 5000
r = 200
G = np.random.randn(d, r)

# Naive computation vs smart computation (proposed in the paper)

Given a full-rank matrix $G \in \mathbb{R}^{d \times r}$, compute $\left(GG^T\right)^{-\frac{1}{2}}$. There are several ways to do it, let's compare their efficiencies.

## 1. Naive

Just compute straightforward.

In [2]:
start_time = time.time()
result_naive = np.linalg.inv(sqrtm(G @ G.T))
print('Naive algorithm finished in {:.4f} seconds'.format(time.time() - start_time))

Naive algorithm finished in 62.3479 seconds


## 2. Naive with SVD

$$
G = U\Sigma V^T
$$
$$
\left(GG^T\right)^{-\frac{1}{2}} = \left(U\Sigma^2U^T\right)^{-\frac{1}{2}} = U\Sigma^{-1}U^T
$$

In [3]:
start_time = time.time()
U, sigma, _ = np.linalg.svd(G @ G.T)
result_naive_svd = U @ (1 / (np.diag(sigma) + eps)) @ U.T
print('Naive algorithm with SVD finished in {:.4f} seconds'.format(time.time() - start_time))

Naive algorithm with SVD finished in 37.1800 seconds


## 3. Smart

$$
\left(GG^T\right)^{-\frac{1}{2}} = G\left(G^TG\right)^{-\frac{3}{2}}G^T
$$

In [4]:
start_time = time.time()
result_smart = G @ (np.linalg.inv(sqrtm(G.T @ G))) ** 3 @ G.T
print('Smart algorithm finished in {:.4f} seconds'.format(time.time() - start_time))

Smart algorithm finished in 0.2099 seconds


## 4. Smart with eigendecomposition

$$
G^TG = V\Sigma^2 V^T
$$
$$
U = GV\Sigma^{-1}
$$
$$
\left(GG^T\right)^{-\frac{1}{2}} = U\Sigma^{-1}U^T
$$

In [5]:
start_time = time.time()
sigma, V = np.linalg.eigh(G.T @ G)
sigma = sigma[::-1]
V = V[::-1]
U = G @ V @ (1 / (np.sqrt(np.diag(sigma)) + eps))
result_smart_eigen = U @ (1 / (np.sqrt(np.diag(sigma)) + eps)) @ U.T
print('Smart algorithm with eigendecomposition finished in {:.4f} seconds'.format(time.time() - start_time))

Smart algorithm with eigendecomposition finished in 0.1879 seconds


So the last method which was proposed in the paper is the most efficient way to perform a desired computation.